# Semantic Search with Recommendations


## Setup

In [1]:
import graphdatascience
graphdatascience.__version__

'1.8'

In [2]:
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime
from datetime import timedelta
from langchain.embeddings import OpenAIEmbeddings, BedrockEmbeddings, SentenceTransformerEmbeddings

In [3]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

In [4]:
load_dotenv('.env', override=True)

True

In [5]:
def load_embedding_model(embedding_model_name: str):
    if embedding_model_name == "openai":
        embeddings = OpenAIEmbeddings()
        dimension = 1536
    elif embedding_model_name == "aws":
        embeddings = BedrockEmbeddings()
        dimension = 1536
    else:
        embeddings = SentenceTransformerEmbeddings(
            model_name="all-MiniLM-L6-v2", cache_folder="/embedding_model"
        )
        dimension = 384
    return embeddings, dimension

In [6]:
embedding_model, dimension = load_embedding_model(os.getenv('EMBEDDING_MODEL'))

In [7]:
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

In [8]:
gds.version()

'2.5.1+39'

## Simple Vector Search

Basic product search using vector index

In [9]:
#search_prompt = 'denim jeans, loose fit, high-waist'
search_prompt = 'winter sweater with zipper'

In [10]:
query_vector = embedding_model.embed_query(search_prompt)
print(f'query vector length: {len(query_vector)}')
print(f'query vector sample: {query_vector[:10]}')

query vector length: 1536
query vector sample: [-0.015721399620501487, -0.01600166252793351, -0.0007557081544117396, -0.020672705397198326, -0.025810854974828365, 0.013906364579883364, -0.009435508570318827, -0.010049416864799713, 0.0030361784703818165, -0.019391505397297122]


In [11]:
res_df = gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.prodName AS prodName,
    product.productTypeName AS productTypeName,
    product.garmentGroupName AS garmentGroupName,
    product.detailDesc AS detailDesc,
    score
    ORDER BY score DESC
''', params={'queryVector': query_vector})

In [12]:
print(f'User Search Prompt: "{search_prompt}"\n\n')
print('Search Results:\n')
res_df

User Search Prompt: "winter sweater with zipper"


Search Results:



,prodName,productTypeName,garmentGroupName,detailDesc,score
0,Catfish Zip,Sweater,Knitwear,"Soft, textured-knit jumper with a ribbed stand-up collar, zip at the top, dropped shoulders and ribbing at the cuffs and hem.",0.933434
1,Yolo Zip L/S,Sweater,Knitwear,"Jumper in a soft, rib knit with a stand-up collar, a visible zip at the front and long sleeves.",0.931782
2,Ben zip hoodie,Sweater,Knitwear,"Fine-knit jacket in a soft viscose blend with a drawstring hood, zip and front pockets. Gently dropped shoulders and ribbing at the cuffs and hem.",0.931016
3,BANANA HALF ZIP SWEATER,Sweater,Knitwear,"Jumper in a soft rib knit with a high stand-up collar, zip at the top, long raglan sleeves with elbow patches, and ribbing at the cuffs and hem.",0.930935
4,Raven Half Zip Sweater,Sweater,Jersey Fancy,"Sports top in stretch, fast-drying functional fabric with a stand-up collar, zip at the front with a chin guard, and a yoke at the back. Low dropped shoulders and long sleeves with thumbholes at the cuffs. Rounded and slightly longer at the back.",0.930436
5,Southern Sweater,Sweater,Knitwear,"Jumper in a soft rib knit with pointelle details, low dropped shoulders and long sleeves. Double ribbed trim around the neckline, and ribbing at the cuffs and hem.",0.929148
6,Zorro half-zip sweater,Sweater,Jersey Basic,"Short top in sweatshirt fabric with a high, ribbed stand-up collar with a zip at the front. Dropped shoulders, long sleeves and ribbing at the cuffs and hem. Soft brushed inside.",0.928683
7,BAY BLOCK STRIPE ZIP-UP,Sweater,Knitwear,"Jumper in a soft, fine-knit modal and cotton blend with a ribbed stand-up collar, zip at the top and long sleeves.",0.928650
8,Yolo Zip LS,Sweater,Knitwear,"Fitted jumper in a soft, rib knit with a turtle neck, visible zip at the top and long sleeves.",0.927658
9,Håkan half zip knit,Sweater,Knitwear,"Jumper in a soft cotton knit with a high, ribbed stand-up collar and zip at the top. Long sleeves, and ribbing at the cuffs and hem.",0.927436


## Recommendation Based on Search Prompt

In [13]:
search_res_df = gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.prodName AS prodName,
    product.productTypeName AS productTypeName,
    product.garmentGroupName AS garmentGroupName,
    product.detailDesc AS detailDesc,
    score
    ORDER BY score DESC
''', params={'queryVector': query_vector})

In [14]:
recommendation_res_df =gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 5, $queryVector)
YIELD node AS search_res_product, score AS search_score
MATCH(search_res_product)<-[:VARIANT_OF]-(a0)-[s:CUSTOMERS_ALSO_PURCHASED]->(a)-[:VARIANT_OF]-(p)
WITH p.prodName AS prodName,
    p.productTypeName AS productTypeName,
    p.garmentGroupName AS garmentGroupName,
    p.detailDesc AS detailDesc,
    sum(s.score) AS aggScore,
    sum(search_score) AS search_score
RETURN prodName, productTypeName, garmentGroupName, search_score*aggScore AS score, detailDesc
    ORDER BY aggScore DESC LIMIT 10
''', params={'queryVector': query_vector})

In [15]:
print(f'User Search Prompt: "{search_prompt}"\n\n')
print('Search Results:\n')
display(search_res_df)
print('\nUser May Also Be Interested In:\n')
display(recommendation_res_df)

User Search Prompt: "winter sweater with zipper"


Search Results:



,prodName,productTypeName,garmentGroupName,detailDesc,score
0,Catfish Zip,Sweater,Knitwear,"Soft, textured-knit jumper with a ribbed stand-up collar, zip at the top, dropped shoulders and ribbing at the cuffs and hem.",0.933434
1,Yolo Zip L/S,Sweater,Knitwear,"Jumper in a soft, rib knit with a stand-up collar, a visible zip at the front and long sleeves.",0.931782
2,Ben zip hoodie,Sweater,Knitwear,"Fine-knit jacket in a soft viscose blend with a drawstring hood, zip and front pockets. Gently dropped shoulders and ribbing at the cuffs and hem.",0.931016
3,BANANA HALF ZIP SWEATER,Sweater,Knitwear,"Jumper in a soft rib knit with a high stand-up collar, zip at the top, long raglan sleeves with elbow patches, and ribbing at the cuffs and hem.",0.930935
4,Raven Half Zip Sweater,Sweater,Jersey Fancy,"Sports top in stretch, fast-drying functional fabric with a stand-up collar, zip at the front with a chin guard, and a yoke at the back. Low dropped shoulders and long sleeves with thumbholes at the cuffs. Rounded and slightly longer at the back.",0.930436
5,Southern Sweater,Sweater,Knitwear,"Jumper in a soft rib knit with pointelle details, low dropped shoulders and long sleeves. Double ribbed trim around the neckline, and ribbing at the cuffs and hem.",0.929148
6,Zorro half-zip sweater,Sweater,Jersey Basic,"Short top in sweatshirt fabric with a high, ribbed stand-up collar with a zip at the front. Dropped shoulders, long sleeves and ribbing at the cuffs and hem. Soft brushed inside.",0.928683
7,BAY BLOCK STRIPE ZIP-UP,Sweater,Knitwear,"Jumper in a soft, fine-knit modal and cotton blend with a ribbed stand-up collar, zip at the top and long sleeves.",0.928650
8,Yolo Zip LS,Sweater,Knitwear,"Fitted jumper in a soft, rib knit with a turtle neck, visible zip at the top and long sleeves.",0.927658
9,Håkan half zip knit,Sweater,Knitwear,"Jumper in a soft cotton knit with a high, ribbed stand-up collar and zip at the top. Long sleeves, and ribbing at the cuffs and hem.",0.927436



User May Also Be Interested In:



,prodName,productTypeName,garmentGroupName,score,detailDesc
0,Niffler Trousers,Trousers,Woven/Jersey/Knitted mix Baby,3.226934,"Pull-on trousers in washed, stretch twill with an elasticated, drawstring waist, front pockets, a fake back pocket and tapered legs."
1,SLIM PULL ON 5 PKT,Trousers,Trousers Denim,3.142746,"Slim-fit, pull-on jeans in washed, superstretch, flexible denim for maximum mobility. Elasticated waist, fake fly with a press-stud, and front and back pockets."
2,Devon basic sweater,Sweater,Knitwear,3.140852,"Jumper in a soft knit with visible seams at the front, dropped shoulders, long sleeves and ribbed roll edges at the cuffs and hem."
3,Sunspot Seamless Crop Top,Vest top,Jersey Fancy,2.885067,"Short, fitted sports top with a racer back and elasticated hem. The sports top is designed with the minimum number of seams for a more comfortable fit and increased mobility."
4,TVP Slim denim trousers,Trousers,Woven/Jersey/Knitted mix Baby,0.807080,"Slim-fit jeans in washed, supersoft, stretch denim with an adjustable, elasticated waist and fly with a press-stud. Fake front pockets, real back pockets and straight legs."
5,Brigita,Sweater,Knitwear,0.805037,"Jumper in a soft, cable knit containing some wool with long raglan sleeves and ribbing around the neckline, cuffs and hem."
6,Don sweater,Sweater,Knitwear,0.800510,"Jumper in a cable-knit cotton blend with long raglan sleeves, buttons on one shoulder and ribbing around the neckline, cuffs and hem."
7,Cool Kite anklet pk,Other accessories,Accessories,0.797286,Shiny metal chain anklets. Adjustable length 21-25 cm.
8,Robbie Tee 9.99:-,T-shirt,Jersey Fancy,0.797215,T-shirt in soft cotton with contrasting colour panels along the shoulders and sleeves.
9,LW (K) CATRIN consc cardi,Cardigan,"Under-, Nightwear",0.796448,"Short cardigan in a soft knit containing some wool with a V-neck, buttons down the front and patch front pockets. Dropped shoulders, long sleeves and ribbing at the cuffs and hem. The polyester content of the cardigan is recycled."


## Personalized Recommendations

In [16]:
CUSTOMER_ID = "daae10780ecd14990ea190a1e9917da33fe96cd8cfa5e80b67b4600171aa77e0"

In [17]:
MAX_DATE = datetime(year=2019, month=1, day=1)

### Example Purchase History

In [18]:
gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a)-[:VARIANT_OF]->(p:Product) WHERE r.tDat > date($cutOffDate)
        RETURN a.articleId AS articleId,
        a.prodName AS prodName,
        r.tDat AS purchaseDate,
        a.productTypeName AS productTypeName,
        p.detailDesc AS detailDesc
        ORDER BY purchaseDate DESC
''', params = {'cutOffDate':str(MAX_DATE  - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID, 'queryVector': query_vector})

,articleId,prodName,purchaseDate,productTypeName,detailDesc
0,753724004,Rosemary,2019-08-05,Dress,"Short dress in woven fabric with 3/4-length sleeves with an opening and ties at the cuffs, and a gently rounded hem. Unlined."
1,733027002,Tove,2019-08-05,Top,"Short top in soft cotton jersey with a round neckline, short sleeves and a seam at the hem with a decorative knot detail at the front."
2,713577001,Malte r-neck,2019-06-27,Sweater,"Jumper in soft, patterned, fine-knit cotton with ribbing around the neckline, cuffs and hem."
3,731142001,Lead Superskinny,2019-06-27,Trousers,"Chinos in stretch twill with a zip fly and button, side pockets, welt back pockets and skinny legs."
4,606711001,Rylee flatform,2019-06-22,Heeled sandals,"Sandals with imitation suede straps, an elastic heel strap and wedge heels. Satin insoles and thermoplastic rubber (TPR) soles. Platform front 2 cm, heel 6 cm."
5,642498007,Bubble Bum Bandeau (1),2019-06-22,Bikini top,"Fully lined bandeau bikini top with padded cups and removable inserts. Detachable ties at the back of the neck, ties at the back, side support and a silicone trim at the top."
6,752193001,Banks,2019-06-22,Hoodie,"Long-sleeved top in sweatshirt fabric made from a cotton blend with a double-layered hood, gently dropped shoulders and ribbing at the cuffs and hem. Soft brushed inside."
7,687016004,DORIS CREW,2019-06-22,Sweater,"Top in sweatshirt fabric with a motif on the front and ribbing around the neckline, cuffs and hem. Soft brushed inside."
8,688537001,Simple as that Cheeky Tanga,2019-06-22,Swimwear bottom,Fully lined bikini bottoms with a mid waist and cutaway coverage at the back.
9,598806011,Dixie tee,2019-02-08,T-shirt,Short top in soft cotton jersey with short sleeves. Contrasting colour trims around the neckline and sleeves.


### Personalized Product Recommendations

In [ ]:
personalized_res_df = gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a0) WHERE r.tDat > date($cutOffDate)
    WITH a0
    MATCH(a0)-[s:CUSTOMERS_ALSO_PURCHASED]->(a)-[:VARIANT_OF]->(p:Product)
    WITH p, sum(s.score) AS aggRecScore
    WITH p, aggRecScore, gds.similarity.cosine($queryVector, p.textEmbedding) AS cosineSimilarity
    RETURN p.productCode AS productCode,
        aggRecScore,
        cosineSimilarity as searchScore,
        p.productTypeName AS productType,
        p.prodName AS name,
        p.detailDesc AS description
        ORDER BY searchScore DESC LIMIT 10
''', params = {'cutOffDate':str(MAX_DATE - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID, 'queryVector': query_vector})

In [ ]:
print(f'User Search Prompt: "{search_prompt}"\n\n')
print('Search Results:\n')
display(search_res_df)
print('\nUser May Also Be Interested In:\n')
display(personalized_res_df)